In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms, models
import os

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
device

device(type='cuda', index=0)

In [10]:
# loading data
data_transforms = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "test": transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = "data/task3/ChestXray"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ["train", "test"]}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ["train", "test"]}
dataset_sizes = {x: len(image_datasets[x]) for x in ["train", "test"]}
class_names = image_datasets["train"].classes


In [11]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2) # 2 classes
model = model.to(device)

In [12]:
criterion = nn.CrossEntropyLoss()
# doing stochastic gradient descent
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [13]:
# training loop
# forward pass
# loss
# backward pass
# tuning
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs - 1}")
        print("-" * 10)

        for phase in ["train", "test"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == "train":
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    return model

model = train_model(model, criterion, optimizer, scheduler, num_epochs=10)


Epoch 0/9
----------
train Loss: 0.3858 Acc: 0.8409
test Loss: 0.2425 Acc: 0.9071
Epoch 1/9
----------
train Loss: 0.2813 Acc: 0.8903
test Loss: 0.5860 Acc: 0.8926
Epoch 2/9
----------
train Loss: 0.2713 Acc: 0.9001
test Loss: 0.6188 Acc: 0.8830
Epoch 3/9
----------
train Loss: 0.2012 Acc: 0.9279
test Loss: 0.2786 Acc: 0.8878
Epoch 4/9
----------
train Loss: 0.1936 Acc: 0.9283
test Loss: 0.2053 Acc: 0.9327
Epoch 5/9
----------
train Loss: 0.1855 Acc: 0.9342
test Loss: 0.2868 Acc: 0.9151
Epoch 6/9
----------
train Loss: 0.1744 Acc: 0.9379
test Loss: 0.2098 Acc: 0.9279
Epoch 7/9
----------
train Loss: 0.1386 Acc: 0.9467
test Loss: 0.2178 Acc: 0.9327
Epoch 8/9
----------
train Loss: 0.1190 Acc: 0.9567
test Loss: 0.2082 Acc: 0.9359
Epoch 9/9
----------
train Loss: 0.1105 Acc: 0.9603
test Loss: 0.2351 Acc: 0.9375


In [14]:
torch.save(model.state_dict(), "resnet50_chest_xray.pth")


In [15]:
# testing from test folder
def test_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

test_accuracy = test_model(model, dataloaders['test'])
print(f'Test Accuracy: {test_accuracy:.4f}')

Test Accuracy: 0.9375


In [16]:
from torchvision import models

# alexnet
alexnet_model = models.alexnet(pretrained=True)

# output class modify
num_features = alexnet_model.classifier[6].in_features
alexnet_model.classifier[6] = nn.Linear(num_features, 2)

# to device
alexnet_model = alexnet_model.to(device)

alexnet_optimizer = optim.SGD(alexnet_model.parameters(), lr=0.001, momentum=0.9)
alexnet_scheduler = lr_scheduler.StepLR(alexnet_optimizer, step_size=7, gamma=0.1)

# using same train loop
alexnet_model = train_model(alexnet_model, criterion, alexnet_optimizer, alexnet_scheduler, num_epochs=10)

torch.save(alexnet_model.state_dict(), "alexnet_chest_xray.pth")


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\adnaa/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:17<00:00, 13.6MB/s] 


Epoch 0/9
----------
train Loss: 0.3909 Acc: 0.8234
test Loss: 0.4356 Acc: 0.7901
Epoch 1/9
----------
train Loss: 0.2993 Acc: 0.8794
test Loss: 0.3495 Acc: 0.8670
Epoch 2/9
----------
train Loss: 0.2488 Acc: 0.8953
test Loss: 0.4142 Acc: 0.8478
Epoch 3/9
----------
train Loss: 0.2441 Acc: 0.9049
test Loss: 0.2783 Acc: 0.9119
Epoch 4/9
----------
train Loss: 0.2255 Acc: 0.9089
test Loss: 0.2182 Acc: 0.9183
Epoch 5/9
----------
train Loss: 0.2145 Acc: 0.9172
test Loss: 0.2608 Acc: 0.9215
Epoch 6/9
----------
train Loss: 0.2037 Acc: 0.9185
test Loss: 0.2891 Acc: 0.8974
Epoch 7/9
----------
train Loss: 0.1486 Acc: 0.9413
test Loss: 0.2084 Acc: 0.9327
Epoch 8/9
----------
train Loss: 0.1542 Acc: 0.9385
test Loss: 0.2024 Acc: 0.9279
Epoch 9/9
----------
train Loss: 0.1439 Acc: 0.9436
test Loss: 0.1802 Acc: 0.9391


In [17]:
# using same test
def test_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

test_accuracy = test_model(alexnet_model, dataloaders['test'])
print(f'Test Accuracy: {test_accuracy:.4f}')


Test Accuracy: 0.9391
